# Inserting to the MySQL database example

In [19]:
import pymysql.cursors
from pymysql.cursors import DictCursor
from datetime import date
from decimal import Decimal
from os import environ as ENV
from dotenv import load_dotenv
import json

## Connect to the database

In [2]:
load_dotenv()

True

In [3]:
conn = pymysql.connect(host=ENV['DB_HOST'],
               user=ENV['DB_USER'],
               password=ENV['DB_PASSWORD'],
               database=ENV['DB_NAME'],
               port=int(ENV['DB_PORT']),
               cursorclass=DictCursor)

## Inserting a set of values into the transaction table

In [18]:
datetime_obj = datetime.strptime("2025-01-13 09:21:00", "%Y-%m-%d %H:%M:%S")
with conn:
    with conn.cursor() as cur:
        query = """
            INSERT INTO FACT_Transaction 
                (transaction_id,
                truck_id,
                payment_method_id,
                total,
                at)
            VALUES 
                (%s, %s, %s, %s, %s);
        """
        cur.execute(query, (2, 4, 1, 10.7, datetime_obj))
    conn.commit()

## Checking insertion and fetch

In [6]:
with conn:
    with conn.cursor() as cur:
        query = "SELECT * FROM transaction_info ORDER BY transaction_id ASC LIMIT 20;"
        cur.execute(query)
        result = cur.fetchall()
    for row in result:
        print(row)

{'transaction_id': 1, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 6)}
{'transaction_id': 2, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('8.40'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 11)}
{'transaction_id': 3, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 21)}
{'transaction_id': 4, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 36)}
{'transaction_id': 5, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 44)}
{'transaction_id': 6, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('10.60'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 59)

## Check query output for daily_report

In [5]:
query1 = """
    SELECT 
        DATE(at) date,
        SUM(total) total
    FROM transaction_info
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY;
"""

query2 = """
    SELECT
        truck_id,
        t.fsa_rating,
        SUM(total) total,
        COUNT(transaction_id) count,
        ROUND(AVG(total),2) avg_amount
    FROM transaction_info
    JOIN DIM_Truck t USING(truck_id)
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY truck_id
    ORDER BY truck_id ASC;
"""

query3 = """
    SELECT
        payment_method,
        SUM(total) total
    FROM transaction_info
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY payment_method;
"""

query4 = """
    SELECT
        truck_id,
        payment_method,
        SUM(total) total,
        ROUND(AVG(total),2) avg_amount
    FROM transaction_info
    WHERE DATE(at) = CURDATE() - INTERVAL 1 DAY
    GROUP BY payment_method, truck_id
    ORDER BY truck_id ASC;
"""

queries = [query1, query2, query3, query4]
titles = ["total_revenue",
          "revenue_by_truck",
          "payment_method",
          "payment_method_by_truck"
          ]

In [32]:
results = {}
with conn.cursor() as cur:
    for i in range(len(queries)):
        cur.execute(queries[i])
        result = cur.fetchall()
        for row in result:
            for key, val in row.items():
                if isinstance(val, date):
                    row[key] = val.strftime('%Y-%m-%d')
                if isinstance(val, Decimal):
                    row[key] = float(val)
        results[titles[i]] = result

[{'date': datetime.date(2025, 1, 28), 'total': Decimal('5018.83')}]
[{'truck_id': 1, 'fsa_rating': 4, 'total': Decimal('2439.30'), 'count': 311, 'avg_amount': Decimal('7.84')}, {'truck_id': 2, 'fsa_rating': 2, 'total': Decimal('700.60'), 'count': 86, 'avg_amount': Decimal('8.15')}, {'truck_id': 3, 'fsa_rating': 5, 'total': Decimal('443.29'), 'count': 71, 'avg_amount': Decimal('6.24')}, {'truck_id': 4, 'fsa_rating': 4, 'total': Decimal('121.63'), 'count': 37, 'avg_amount': Decimal('3.29')}, {'truck_id': 5, 'fsa_rating': 4, 'total': Decimal('388.60'), 'count': 70, 'avg_amount': Decimal('5.55')}, {'truck_id': 6, 'fsa_rating': 3, 'total': Decimal('925.41'), 'count': 160, 'avg_amount': Decimal('5.78')}]
[{'payment_method': 'card', 'total': Decimal('2527.93')}, {'payment_method': 'cash', 'total': Decimal('2490.90')}]
[{'truck_id': 1, 'payment_method': 'cash', 'total': Decimal('1315.80'), 'avg_amount': Decimal('7.93')}, {'truck_id': 1, 'payment_method': 'card', 'total': Decimal('1123.50'), 'a

In [33]:
results

{'total_revenue': [{'date': '2025-01-28', 'total': 5018.83}],
 'revenue_by_truck': [{'truck_id': 1,
   'fsa_rating': 4,
   'total': 2439.3,
   'count': 311,
   'avg_amount': 7.84},
  {'truck_id': 2,
   'fsa_rating': 2,
   'total': 700.6,
   'count': 86,
   'avg_amount': 8.15},
  {'truck_id': 3,
   'fsa_rating': 5,
   'total': 443.29,
   'count': 71,
   'avg_amount': 6.24},
  {'truck_id': 4,
   'fsa_rating': 4,
   'total': 121.63,
   'count': 37,
   'avg_amount': 3.29},
  {'truck_id': 5,
   'fsa_rating': 4,
   'total': 388.6,
   'count': 70,
   'avg_amount': 5.55},
  {'truck_id': 6,
   'fsa_rating': 3,
   'total': 925.41,
   'count': 160,
   'avg_amount': 5.78}],
 'payment_method': [{'payment_method': 'card', 'total': 2527.93},
  {'payment_method': 'cash', 'total': 2490.9}],
 'payment_method_by_truck': [{'truck_id': 1,
   'payment_method': 'cash',
   'total': 1315.8,
   'avg_amount': 7.93},
  {'truck_id': 1,
   'payment_method': 'card',
   'total': 1123.5,
   'avg_amount': 7.75},
  {'tr

In [34]:
with open('daily_report_data.json', 'w') as f:
    json.dump(results, f, indent=4)